In [1]:
%load_ext autoreload

%autoreload 2
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
from data import get_datasets
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from data import get_datasets
from experiment import create_model, evaluate, find_adv_k, sentiment
import model_adv
from bertviz import head_view
import torch

In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [3]:
MAX_LENGTH = 200
BATCH_SIZE = 1

(train_dataset, test_dataset), info = get_datasets(max_length=MAX_LENGTH, batch_size=BATCH_SIZE)

LR = 0.0001
VOCAB_SIZE = info.features['text'].encoder.vocab_size 
POS_ENC = True
LOAD_CHECKPOINT = True

In [4]:
transformer, optimizer, checkpoint, manager = create_model(models_dir="models_entmax", load_checkpoint=LOAD_CHECKPOINT, 
                           vocab_size=VOCAB_SIZE, use_positional_encoding=POS_ENC, run_eagerly=True, learning_rate=LR)

Loaded previously trained model.


In [5]:
sentence = "This is a very terrible movie . I will never watch this bad movie again ."
sentence = "This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here."
sent_original, weights_original, tokens = sentiment(sentence, info.features['text'].encoder, transformer, use_sparsemax=True)

print(f"Initial sentiment: {sent_original}")

Initial sentiment: neg


## Entmax

In [6]:
# Visualize
head_view(torch.tensor(weights_original.numpy()[0]),tokens)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>